Importing pandas ; Importing monthly progress error by book value ; Importing monthly progress by book value csv ; Importing matplotlib

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
df_progress=pd.read_csv('../../data/output/monthly-progress-by-book-value.csv',sep=';')
df_error=pd.read_csv('../../data/output/monthly-progress-by-book-value-error.csv',sep=';')
print(df_progress.head())
print(df_error.head())

   Year  Month  Progress  2 Mean  3 Mean  4 Mean  5 Mean
0  2012      1  0.825456     NaN     NaN     NaN     NaN
1  2012      2  0.783939     NaN     NaN     NaN     NaN
2  2012      3  0.802500     NaN     NaN     NaN     NaN
3  2012      4  0.814931     NaN     NaN     NaN     NaN
4  2012      5  0.785893     NaN     NaN     NaN     NaN
   Year  Month  2 Error  3 Error  4 Error  5 Error
0  2012      1      NaN      NaN      NaN      NaN
1  2012      2      NaN      NaN      NaN      NaN
2  2012      3      NaN      NaN      NaN      NaN
3  2012      4      NaN      NaN      NaN      NaN
4  2012      5      NaN      NaN      NaN      NaN


Calculating the Alpha value

In [52]:
df_temp=df_error.copy()

for i,col in enumerate(df_temp.columns[2:]):
    df_temp[str(i+2)+' Ratio']=df_temp[col]/df_temp[col].shift(1)
    
print(df_temp[df_temp['Year']==2018])
print(df_temp.mean())


    Year  Month   2 Error   3 Error   4 Error   5 Error   2 Ratio   3 Ratio  \
72  2018      1  0.050105  0.052729  0.050117  0.048906  0.169379  0.280316   
73  2018      2  0.096651  0.082046  0.074185  0.056470  1.928983  1.555991   
74  2018      3  0.051991  0.034026  0.026573  0.005077  0.537923  0.414720   
75  2018      4  0.011486 -0.004341 -0.016381 -0.032971  0.220915 -0.127577   
76  2018      5 -0.026391 -0.019144 -0.037860 -0.054863 -2.297760  4.410115   
77  2018      6 -0.042874 -0.026348 -0.043458 -0.037279  1.624571  1.376271   
78  2018      7 -0.076440 -0.040042 -0.056291 -0.050760  1.782882  1.519778   
79  2018      8 -0.057185  0.003647 -0.015712 -0.002908  0.748112 -0.091069   
80  2018      9 -0.063479  0.020243 -0.015609 -0.001932  1.110061  5.551268   
81  2018     10 -0.136178 -0.067158 -0.094606 -0.084918  2.145235 -3.317541   
82  2018     11 -0.078365 -0.010351 -0.046568 -0.028959  0.575459  0.154125   
83  2018     12 -0.034046  0.035269 -0.004022  0.025

Calculating manual autoregressive model with y(t)(new)=y(t)+y(t-1)xAlpha

In [53]:
alpha=0

years=df_progress['Year'].unique()
months=df_progress['Month'].unique()

df_progress=df_progress.set_index(['Year','Month'])
df_error=df_error.set_index(['Year','Month'])

for year in years:
    for i,col in enumerate(df_error.columns):
        df_progress[str(i+2)+' AR']=df_progress[str(i+2)+' Mean']+(df_error[col].shift(1)*alpha*df_progress[str(i+2)+' Mean'])
        df_progress.loc[(year,1),str(i+2)+' AR']=df_progress.loc[(year,1),str(i+2)+' Mean']
        df_progress[str(i+2)+' AR Error']=(df_progress[str(i+2)+' AR']-df_progress['Progress'])/df_progress['Progress']

print(df_progress.abs().mean())
        


Progress      0.908181
2 Mean        0.903208
3 Mean        0.902727
4 Mean        0.903064
5 Mean        0.904144
2 AR          0.903853
2 AR Error    0.088762
3 AR          0.903435
3 AR Error    0.075872
4 AR          0.903893
4 AR Error    0.076638
5 AR          0.905108
5 AR Error    0.073526
dtype: float64


Calculating error ARIMA model to actual progress